In [1]:
import numpy as np
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import matplotlib.pyplot as plt

# learning rate decay
from torch.optim import lr_scheduler

batch_size = 16
learning_rate=0.001
num_epoch=1

In [2]:
cifar_train=dset.CIFAR10("CIFAR10/", train=True, transform=transforms.Compose([
                                                    transforms.ToTensor(),
                                                    transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5)),
                                                  ])
                                                  , target_transform=None, download=True)  # data normalization

cifar_test=dset.CIFAR10("CIFAR10/", train=False, transform=transforms.Compose([
                                                    transforms.ToTensor(),
                                                    transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5)),
                                                  ])
                                                  , target_transform=None, download=True)  # data normalization


Extracting CIFAR10/cifar-10-python.tar.gz to CIFAR10/
Files already downloaded and verified


In [3]:
def ComputeAccr(dloader, imodel):
    correct = 0
    total = 0
    with torch.no_grad():
      for j, [imgs, labels] in enumerate(dloader): # batch_size 만큼
          img = Variable(imgs).cuda() # x
          # img = Variable(img, requires_grad=False).cuda()
          label = Variable(labels).cuda() # y # gpu 가속        
          
          output = imodel(img) # forward propagation
          _, output_index = torch.max(output, 1) 
          
          total += label.size(0)
          correct += (output_index ==label).sum().float() # 예상값 == 실제값
    print("Accuracy of Test Data: {}".format(100*correct/total))

In [4]:
# cifar_train 을 load
train_loader = torch.utils.data.DataLoader(list(cifar_train)[:], batch_size=batch_size, shuffle=True, num_workers=2,drop_last=True)

# cifar_test 을 load
test_loader = torch.utils.data.DataLoader(cifar_test, batch_size=batch_size, shuffle=False, num_workers=2,drop_last=True)

In [5]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN,self).__init__()
    self.layer=nn.Sequential(
        nn.Conv2d(3,16,3,padding=1),
        #nn.ReLU(),
        nn.ELU(),
        nn.Dropout2d(0.1),
        nn.BatchNorm2d(16),
        nn.Conv2d(16,32,3,padding=1),
        #nn.ReLU(),
        nn.ELU(),
        nn.Dropout2d(0.1),
        nn.BatchNorm2d(32),       
        nn.MaxPool2d(2,2),
        nn.Conv2d(32,64,3,padding=1),
        #nn.ReLU(),
        nn.ELU(),
        nn.Dropout2d(0.1),
        nn.BatchNorm2d(64),
        nn.MaxPool2d(2,2)
    )
    self.fc_layer=nn.Sequential(
        nn.Linear(64*8*8, 100),
        #nn.ReLU(),
        nn.ELU(),
        nn.Dropout2d(0.4),
        nn.BatchNorm1d(100),
        nn.Linear(100, 50),
        #nn.ReLU(),
        nn.ELU(),
        nn.Dropout2d(0.4),
        nn.BatchNorm1d(50),
        nn.Linear(50,10)
        #nn.Linear(100, 10)
    )
  
  # weight intitialization
    for m in self.modules():
      if isinstance(m,nn.Conv2d):
        init.kaiming_normal_(m.weight.data)
        m.bias.data.fill_(0)
      if isinstance(m, nn.Linear):
        init.kaiming_normal_(m.weight.data)
        m.bias.data.fill_(0)

  def forward(self,x):
    out=self.layer(x)
    out=out.view(batch_size,-1)
    out=self.fc_layer(out)

    return out

model=CNN().cuda()

In [6]:
# 5. loss, optimizer
loss_func = nn.CrossEntropyLoss() # logit(# of classes), target(1)
#optimizer = optim.SGD(model.parameters(), lr = learning_rate)

optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate) # Adam optimizer

In [ ]:
# 삭제된후 3번째 시도..
#6. 학습
model.train()
num_epoch = 40
scheduler = lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.2) # learning rate decay
for i in range(num_epoch):
    for j, [image, label] in enumerate(train_loader):
        x=Variable(image).cuda()
        y_=Variable(label).cuda()

        optimizer.zero_grad()
        output = model.forward(x) # forward propagation
        loss=loss_func(output, y_) # logit(#of cloasses), target(1)
        loss.backward() # back propagation
        optimizer.step() # weight 조정        
        if j%1000==0:
            print(j,loss)
    print(i)

0 tensor(0.7680, device='cuda:0', grad_fn=<NllLossBackward>)
1000 tensor(0.5635, device='cuda:0', grad_fn=<NllLossBackward>)
2000 tensor(0.4829, device='cuda:0', grad_fn=<NllLossBackward>)
3000 tensor(0.6867, device='cuda:0', grad_fn=<NllLossBackward>)
0
0 tensor(0.7522, device='cuda:0', grad_fn=<NllLossBackward>)
1000 tensor(0.9677, device='cuda:0', grad_fn=<NllLossBackward>)
2000 tensor(0.5008, device='cuda:0', grad_fn=<NllLossBackward>)
3000 tensor(0.6658, device='cuda:0', grad_fn=<NllLossBackward>)
1
0 tensor(0.7059, device='cuda:0', grad_fn=<NllLossBackward>)
1000 tensor(0.2020, device='cuda:0', grad_fn=<NllLossBackward>)
2000 tensor(0.5198, device='cuda:0', grad_fn=<NllLossBackward>)
3000 tensor(0.7012, device='cuda:0', grad_fn=<NllLossBackward>)
2
0 tensor(0.9553, device='cuda:0', grad_fn=<NllLossBackward>)
1000 tensor(0.5002, device='cuda:0', grad_fn=<NllLossBackward>)
2000 tensor(0.6048, device='cuda:0', grad_fn=<NllLossBackward>)
3000 tensor(0.4826, device='cuda:0', grad_fn=<

learning rate 0.001 / 0.005 확인

In [ ]:
model.eval()
ComputeAccr(test_loader, model)
netname = './nets/net050_g+A40+lr0.001.pkl'
torch.save(model, netname, )

Accuracy of Test Data: 76.16999816894531


In [ ]:
model.eval()
ComputeAccr(test_loader, model)
netname = './nets/net051_g+A40+lr0.005.pkl'
torch.save(model, netname, )

Accuracy of Test Data: 75.56999969482422


ELU 시도 - (dropout, dataAugmentation제외) - epoch 30

In [ ]:
model.eval()
ComputeAccr(test_loader, model)
netname = './nets/net061_ELU30.pkl'
torch.save(model, netname, )

Accuracy of Test Data: 74.11000061035156


ELU 시도 - 모두+dropout - epoch 30

In [ ]:
model.eval()
ComputeAccr(test_loader, model)
netname = './nets/net062_ELU30+drop.pkl'
torch.save(model, netname, )

Accuracy of Test Data: 76.31999969482422


ELU 시도 - 모두+drop+dataAug - epoch 30--> dataAug는 적용하지 않도록함

In [ ]:
model.eval()
ComputeAccr(test_loader, model)
netname = './nets/net062_ELU30+drop+aug.pkl'
torch.save(model, netname, )

Accuracy of Test Data: 27.799999237060547


ELU 시도 - 모두+ layer별 dropout조정- epoch 30 --> 0.1/0.1/0.1/0.4 확정

In [ ]:
model.eval()
ComputeAccr(test_loader, model)
netname = './nets/net063_ELU30+drop(1234).pkl'
torch.save(model, netname, )

Accuracy of Test Data: 76.79000091552734


In [ ]:
model.eval()
ComputeAccr(test_loader, model)
netname = './nets/net063_ELU30+drop(1235).pkl'
torch.save(model, netname, )

Accuracy of Test Data: 76.11000061035156


In [ ]:
model.eval()
ComputeAccr(test_loader, model)
netname = './nets/net063_ELU30+drop(11524).pkl'
torch.save(model, netname, )

Accuracy of Test Data: 77.0199966430664


In [ ]:
model.eval()
ComputeAccr(test_loader, model)
netname = './nets/net063_ELU30+drop(1114).pkl'
torch.save(model, netname, )

Accuracy of Test Data: 77.05999755859375


In [ ]:
model.eval()
ComputeAccr(test_loader, model)
netname = './nets/net063_ELU30+drop(.5.5.54).pkl'
torch.save(model, netname, )

Accuracy of Test Data: 77.1500015258789


In [ ]:
model.eval()
ComputeAccr(test_loader, model)
netname = './nets/net063_ELU30+drop(XXX4).pkl'
torch.save(model, netname, )

Accuracy of Test Data: 76.44999694824219


In [ ]:
model.eval()
ComputeAccr(test_loader, model)
netname = './nets/net063_ELU30+drop(.2.2.24).pkl'
torch.save(model, netname, )

Accuracy of Test Data: 76.50999450683594


In [ ]:
model.eval()
ComputeAccr(test_loader, model)
netname = './nets/net063_ELU30+drop(1114)_again4,,,.pkl'
torch.save(model, netname, )

Accuracy of Test Data: 77.68000030517578


drop(1114)확정 epoch10/15/40 시도 --> epoch 30확정

In [ ]:
model.eval()
ComputeAccr(test_loader, model)
netname = './nets/net064_ELU10(1114).pkl'
torch.save(model, netname, )

Accuracy of Test Data: 75.61000061035156


In [ ]:
model.eval()
ComputeAccr(test_loader, model)
netname = './nets/net064_ELU20(1114).pkl'
torch.save(model, netname, )

Accuracy of Test Data: 77.56999969482422


In [ ]:
model.eval()
ComputeAccr(test_loader, model)
netname = './nets/net064_ELU40(5554).pkl'
torch.save(model, netname, )

Accuracy of Test Data: 77.3499984741211


epoch30, dropout1114, ELU + batch32

In [ ]:
model.eval()
ComputeAccr(test_loader, model)
netname = './nets/net065_ELU+b32.pkl'
torch.save(model, netname, )

Accuracy of Test Data: 76.25200653076172


dropout1114, ELU + fc_layer 추가 - epoch30/40

In [ ]:
model.eval()
ComputeAccr(test_loader, model)
netname = './nets/net065_ELU+fclayer!.pkl'
torch.save(model, netname, )

Accuracy of Test Data: 77.08999633789062


**가장 높은 정확도**

In [ ]:
model.eval()
ComputeAccr(test_loader, model)
netname = './nets/net065_40ELU+fclayer!.pkl'
torch.save(model, netname, )


Accuracy of Test Data: 78.13999938964844


In [7]:
netname='./nets/net065_40ELU+fclayer!.pkl'
model=torch.load(netname)

ComputeAccr(test_loader, model)

Accuracy of Test Data: 78.13999938964844
